In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statistics 
import math

In [ ]:
matches = pd.read_csv("/content/sample_data/matches.csv")
matches.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
# setting up the table with relevant columns
dropList = ["result","player_of_match","venue","umpire1","umpire2","umpire3"]
matches.drop(labels=dropList, axis=1, inplace=True)
matches.head() 

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Sunrisers Hyderabad,35,0
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,0,Rising Pune Supergiant,0,7
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,0,Kolkata Knight Riders,0,10
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,6
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,0,Royal Challengers Bangalore,15,0


In [ ]:
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
300,301,2011,Delhi,2011-05-21,Delhi Daredevils,Pune Warriors,Delhi Daredevils,bat,0,NaN,0,0
545,546,2015,Bangalore,2015-04-29,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0
570,571,2015,Bangalore,2015-05-17,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,NaN,0,0
744,11340,2019,Bengaluru,30/04/19,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,0,NaN,0,0


In [ ]:
matches['winner'].fillna('Draw', inplace=True)
matches[pd.isnull(matches['winner'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [ ]:
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets
461,462,2014,NaN,2014-04-19,Mumbai Indians,Royal Challengers Bangalore,Royal Challengers Bangalore,field,0,Royal Challengers Bangalore,0,7
462,463,2014,NaN,2014-04-19,Kolkata Knight Riders,Delhi Daredevils,Kolkata Knight Riders,bat,0,Delhi Daredevils,0,4
466,467,2014,NaN,2014-04-23,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,field,0,Chennai Super Kings,7,0
468,469,2014,NaN,2014-04-25,Sunrisers Hyderabad,Delhi Daredevils,Sunrisers Hyderabad,bat,0,Sunrisers Hyderabad,4,0
469,470,2014,NaN,2014-04-25,Mumbai Indians,Chennai Super Kings,Mumbai Indians,bat,0,Chennai Super Kings,0,7
474,475,2014,NaN,2014-04-28,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,field,0,Kings XI Punjab,0,5
476,477,2014,NaN,2014-04-30,Sunrisers Hyderabad,Mumbai Indians,Mumbai Indians,field,0,Sunrisers Hyderabad,15,0


In [ ]:
matches['city'].fillna('Dubai', inplace=True)
matches[pd.isnull(matches['city'])]

,id,season,city,date,team1,team2,toss_winner,toss_decision,dl_applied,winner,win_by_runs,win_by_wickets


In [ ]:
matches.replace(["Deccan Chargers","Delhi Daredevils"],["Sunrisers Hyderabad","Delhi Capitals"],inplace=True,regex=True)
match1 = matches[((matches.team1=="Sunrisers Hyderabad") & (matches.team2=="Delhi Capitals")) | ((matches.team1=="Delhi Capitals") & (matches.team2=="Sunrisers Hyderabad"))]
match1.shape[0]

26

In [ ]:
mw_dc = 0 
mw_srh = 0
lst= [i for i in match1['winner']] 
print("Win Tracker!")
for i in lst:
  if i=="Sunrisers Hyderabad":
    mw_srh += 1
  elif i=='Draw':
    continue
  else:
    mw_dc += 1
  print(str(mw_dc)+" "+str(mw_srh))
print("DC vs SRH : "+str(mw_dc)+" "+str(mw_srh))

Win Tracker!
0 1
1 1
2 1
3 1
4 1
5 1
5 2
5 3
5 4
5 5
6 5
7 5
8 5
8 6
8 7
8 8
8 9
9 9
9 10
10 10
11 10
11 11
11 12
11 13
12 13
13 13
DC vs SRH : 13 13


In [ ]:
last_3_season = matches[(matches['season']>=2017) & np.logical_or(np.logical_and(matches.team1=="Sunrisers Hyderabad",matches.team2=="Delhi Capitals"),np.logical_and(matches.team1=="Delhi Capitals",matches.team2=="Sunrisers Hyderabad"))]
last_3_season.groupby('winner').season.count()

winner
Delhi Capitals         3
Sunrisers Hyderabad    4
Name: season, dtype: int64

Out of 26 matches held between SRH and DC, Both are equal against each other with 13 is to 13.But in the last three seasons, SRH has got edge lead of victories over DC i.e 4 is to 3

In [ ]:
deliveries = pd.read_csv("/content/sample_data/deliveries.csv")
deliveries.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
dropToBeList = ['inning','is_super_over','bye_runs','legbye_runs','fielder']
deliveries.drop(dropToBeList, axis=1, inplace=True)
deliveries.replace(['Deccan Chargers','Delhi Daredevils'],['Sunrisers Hyderabad','Delhi Capitals'],inplace=True,regex=True)
deliveries['dismissal_kind'].fillna('Not Out',inplace=True)
deliveries.head()

,match_id,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,wide_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,4,0,4,NaN,Not Out
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,NaN,Not Out
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,2,0,0,0,2,2,NaN,Not Out


In [ ]:
ballbyball = deliveries[((deliveries.batting_team=="Delhi Capitals") & (deliveries.bowling_team=="Sunrisers Hyderabad")) | ((deliveries.batting_team=="Sunrisers Hyderabad") & (deliveries.bowling_team=="Delhi Capitals"))]
no_of_matches=list(set([i for i in ballbyball['match_id']]))
print(len(no_of_matches))

26


In [ ]:
#Q2
total_runs_per_match = ballbyball.groupby('match_id').total_runs.sum().reset_index()
total_runs_per_match['total_runs'].describe()

count     26.000000
mean     313.538462
std       55.685711
min      161.000000
25%      287.750000
50%      320.000000
75%      350.750000
max      383.000000
Name: total_runs, dtype: float64

There may be the total_score of 320-350 runs between these two teams!

In [ ]:
#Q3
lst=['caught and bowled','caught']
total_caught_dismissals = ballbyball[ballbyball['dismissal_kind'].isin(lst)].groupby('match_id').dismissal_kind.count()
total_caught_dismissals.describe()

count    26.000000
mean      7.192308
std       2.623269
min       3.000000
25%       5.000000
50%       7.000000
75%       8.750000
max      13.000000
Name: dismissal_kind, dtype: float64

There might be 7-9 catches present in the match between SRH and DC

In [47]:
#Q4
srh_score = ballbyball[(ballbyball['batting_team']=='Sunrisers Hyderabad') & (ballbyball['over']>=1) & (ballbyball['over']<=5)].groupby('match_id').total_runs.sum().reset_index()
dc_score = ballbyball[(ballbyball['batting_team']=='Delhi Capitals') & (ballbyball['over']>=1) & (ballbyball['over']<=5)].groupby('match_id').total_runs.sum().reset_index()
srh_scores=[i for i in srh_score['total_runs']]
dc_scores=[i for i in dc_score['total_runs']]
diff=[]
for i in range(len(srh_scores)):
  diff.append(abs(dc_scores[i]-srh_scores[i]))
print(statistics.mean(diff))
print(statistics.median(diff))
print(statistics.median_high(diff))


8.73076923076923
6.0
6


SRH usually takes 6 to 9 balls more or less runs in powerplay compared to DC

In [51]:
#Q5
total_extras_per_match = ballbyball[(ballbyball.extra_runs!=0)].groupby('match_id').extra_runs.count().reset_index()
total_extras_per_match['extra_runs'].describe()

count    26.000000
mean     10.846154
std       3.402262
min       5.000000
25%       8.250000
50%      10.500000
75%      12.750000
max      20.000000
Name: extra_runs, dtype: float64

We can expect 10 to 13 extra runs in these SRH versus DC matches!